# Testing

---
## Dice score using nnunet

### Bone segmentation

#### Total spine segmentation

In [ ]:
import multiprocessing
import os
from os import listdir
from os.path import join
from copy import deepcopy
from multiprocessing import Pool
import numpy as np
from nnunetv2.imageio.simpleitk_reader_writer import SimpleITKIO
from batchgenerators.utilities.file_and_folder_operations import subfiles, join, save_json, load_json, \
    isfile
from nnunetv2.utilities.json_export import recursive_fix_for_json_export
import sys
sys.path.append(".")
from test_utils import compute_metrics_on_folder

##### REAL

In [ ]:
## General
test_folder = "../../nnUNet/nnUNet_test"
output_folder = "./metrics/DSC"
# Label specific
regions_or_labels = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26]
folder_ref="../../nnUNet/nnUNet_raw/Dataset100_HnN/labelsTs"
# Experiment
dataset_id = "Dataset100_HnN"

for sub_folder in listdir(join(test_folder, dataset_id)):
    folder_pred=join(test_folder, dataset_id, sub_folder)
    output_file= join(output_folder, dataset_id, sub_folder, "summary.json")
    os.makedirs(join(output_folder, dataset_id, sub_folder), exist_ok=True)
    if os.path.exists(output_file):
        continue
    try:
        compute_metrics_on_folder(folder_ref=folder_ref, 
                                folder_pred=folder_pred, 
                                output_file=output_file,
                                image_reader_writer=SimpleITKIO(),
                                file_ending='.nii.gz',
                                regions_or_labels=regions_or_labels,
                                ignore_label=None,
                                num_processes=8,
                                chill=False)
        print(join(output_folder, dataset_id, sub_folder, "summary.json"))
    except:
        print(f"Problems with: {join(dataset_id, sub_folder, 'summary.json')}")

print("DONE")

##### FAKE

In [ ]:
## General
test_folder = "../../nnUNet/nnUNet_test"
output_folder = "./metrics/DSC"
# Label specific
regions_or_labels = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26]
folder_ref="../../nnUNet/nnUNet_raw/Dataset100_HnN/labelsTs"
# Experiment
dataset_ids_list = ["Dataset101_HnN", "Dataset102_HnN", "Dataset103_HnN", "Dataset104_HnN", "Dataset105_HnN", "Dataset111_HnN", "Dataset112_HnN", "Dataset113_HnN", "Dataset114_HnN", "Dataset115_HnN"]

for dataset_id in dataset_ids_list:
    for sub_folder in listdir(join(test_folder, dataset_id)):
        folder_pred=join(test_folder, dataset_id, sub_folder)
        output_file= join(output_folder, dataset_id, sub_folder, "summary.json")
        os.makedirs(join(output_folder, dataset_id, sub_folder), exist_ok=True)
        if os.path.exists(output_file):
            continue
        try:
            compute_metrics_on_folder(folder_ref=folder_ref, 
                                    folder_pred=folder_pred, 
                                    output_file=output_file,
                                    image_reader_writer=SimpleITKIO(),
                                    file_ending='.nii.gz',
                                    regions_or_labels=regions_or_labels,
                                    ignore_label=None,
                                    num_processes=8,
                                    chill=False)
            print(join(output_folder, dataset_id, sub_folder, "summary.json"))
        except:
            print(f"Problems with: {join(dataset_id, sub_folder, 'summary.json')}")

print("DONE")

##### Semantic segmentation evaluation

In [ ]:
import json
import numpy as np
import nibabel as nib
from os import listdir
from os.path import join
from multiprocessing import Pool, cpu_count

def get_semantic_seg_regions(pred_file_data, vertebrae, discs, spinal_canal, spinal_cord):
    return [
        np.isin(pred_file_data, vertebrae).astype(np.uint8),
        np.isin(pred_file_data, discs).astype(np.uint8),
        np.isin(pred_file_data, spinal_canal).astype(np.uint8),
        np.isin(pred_file_data, spinal_cord).astype(np.uint8)
    ]

def get_TP_TN_FP_FN(ground_truth, prediction):
    TP = np.sum((ground_truth == 1) & (prediction == 1))
    TN = np.sum((ground_truth == 0) & (prediction == 0))
    FP = np.sum((ground_truth == 0) & (prediction == 1))
    FN = np.sum((ground_truth == 1) & (prediction == 0))
    return TP, TN, FP, FN

def compute_dsc(TP, FP, FN):
    return (2 * TP) / (2 * TP + FP + FN) if (2 * TP + FP + FN) > 0 else 0.0

def process_file(predictions_folder, pred_file_name, gt_folder, vertebrae, discs, spinal_canal, spinal_cord):
    gt_file_data = nib.load(join(gt_folder, pred_file_name)).get_fdata()
    pred_file_data = nib.load(join(predictions_folder, pred_file_name)).get_fdata()
    
    gt_regions = get_semantic_seg_regions(gt_file_data, vertebrae, discs, spinal_canal, spinal_cord)
    pred_regions = get_semantic_seg_regions(pred_file_data, vertebrae, discs, spinal_canal, spinal_cord)
    
    region_names = ["Vertebrae", "Discs", "Spinal Canal", "Spinal Cord"]
    
    file_results = {}
    for name, gt, pred in zip(region_names, gt_regions, pred_regions):
        TP, TN, FP, FN = get_TP_TN_FP_FN(gt, pred)
        file_results[name] = {"TP": TP, "TN": TN, "FP": FP, "FN": FN}
    
    return file_results

def compute_mean_dsc_folder_totalspineseg(gt_folder, predictions_folder, output_file):
    vertebrae = [12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
    discs = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
    spinal_canal = [25]
    spinal_cord = [26]
    
    pred_files = [f for f in listdir(predictions_folder) if f.endswith('nii.gz')]
    
    with Pool(cpu_count()) as pool:
        results = pool.starmap(
            process_file,
            [(predictions_folder, f, gt_folder, vertebrae, discs, spinal_canal, spinal_cord) for f in pred_files]
        )
    
    aggregated_metrics = {name: {"TP": 0, "TN": 0, "FP": 0, "FN": 0} for name in ["Vertebrae", "Discs", "Spinal Canal", "Spinal Cord"]}
    
    for file_result in results:
        for region, metrics in file_result.items():
            for key in ["TP", "TN", "FP", "FN"]:
                aggregated_metrics[region][key] += metrics[key]
    
    dsc_results = {region: compute_dsc(metrics["TP"], metrics["FP"], metrics["FN"]) for region, metrics in aggregated_metrics.items()}
    
    with open(output_file, 'w') as f:
        json.dump(dsc_results, f, indent=4)
    
    print(f"Mean DSC results saved to {output_file}")


In [ ]:
import os
## General
test_folder = "../../nnUNet/nnUNet_test"
output_folder = "./metrics/DSC"
# Label specific
gt_folder="../../nnUNet/nnUNet_raw/Dataset100_HnN/labelsTs"
# Experiment
dataset_ids_list = ["Dataset100_HnN", "Dataset101_HnN", "Dataset102_HnN", "Dataset103_HnN", "Dataset104_HnN", "Dataset105_HnN", "Dataset111_HnN", "Dataset112_HnN", "Dataset113_HnN", "Dataset114_HnN", "Dataset115_HnN"]

for dataset_id in dataset_ids_list:
    for sub_folder in listdir(join(test_folder, dataset_id)):
        folder_pred=join(test_folder, dataset_id, sub_folder)
        output_file= join(output_folder, dataset_id, sub_folder, "summary_semantic.json")
        os.makedirs(join(output_folder, dataset_id, sub_folder), exist_ok=True)
        if os.path.exists(output_file):
            continue
        
        compute_mean_dsc_folder_totalspineseg(gt_folder=gt_folder, predictions_folder=folder_pred, output_file=output_file)
        print(join(output_folder, dataset_id, sub_folder, "summary_semantic.json"))
        

print("DONE")

##### Instance segmentation evaluation

In [ ]:
# MEAN DSC
import json
import pandas as pd
from os import listdir
from os.path import join
epsilon=1e-6

problems = []
vertebrae = [12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
discs = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
spinal_canal = [25]
spinal_cord = [26]

structure_name_list = ['Vertebrae', 'Discs', 'Spinal Canal', 'Spinal Cord']

import os
## General
test_folder = "../../nnUNet/nnUNet_test"
output_folder = "./metrics/DSC"
# Label specific
gt_folder="../../nnUNet/nnUNet_raw/Dataset100_HnN/labelsTs"
# Experiment
dataset_ids_list = ["Dataset100_HnN", "Dataset101_HnN", "Dataset102_HnN", "Dataset103_HnN", "Dataset104_HnN", "Dataset105_HnN", "Dataset111_HnN", "Dataset112_HnN", "Dataset113_HnN", "Dataset114_HnN", "Dataset115_HnN"]

for dataset_id in dataset_ids_list:
    for sub_folder in listdir(join(test_folder, dataset_id)):
        folder_pred=join(test_folder, dataset_id, sub_folder)
        output_file= join(output_folder, dataset_id, sub_folder, "summary_instance.json")
        os.makedirs(join(output_folder, dataset_id, sub_folder), exist_ok=True)
        if os.path.exists(output_file):
            continue

        sumary_json_file = f"./metrics/DSC/{dataset_id}/{sub_folder}/summary.json"
        if not os.path.exists(sumary_json_file):
            problems.append(sumary_json_file)
            continue

        
        else:
            with open(sumary_json_file, 'r') as file:
                # Load the JSON data
                data = json.load(file)
                dsc_results = {}
                for idx_structure, structure in enumerate([vertebrae, discs, spinal_canal, spinal_cord]):
                    n_TP = 0
                    n_FP = 0
                    n_FN = 0
                    for case in data['metric_per_case']: # go to all predictions
                        for label_n in structure: # To all labels
                            if str(label_n) in case['metrics']:
                                n_TP += case['metrics'][str(label_n)]['TP']
                                n_FP += case['metrics'][str(label_n)]['FP']
                                n_FN += case['metrics'][str(label_n)]['FN']
                                
                    dsc = (2 * n_TP ) / (2 * n_TP + n_FP + n_FN )
                    dsc_results[structure_name_list[idx_structure]] = dsc
            
            with open(output_file, 'w') as f:
                json.dump(dsc_results, f, indent=4)
            
            print(f"Mean DSC results saved to {output_file}")
        
  
print(f"problems:")
for problem in problems:
    print(problem)


##### Make a excel sheet resuming

In [ ]:
dsc_folder = "./metrics/DSC"
dataset_ids_list = ["Dataset100_HnN", "Dataset101_HnN", "Dataset102_HnN", "Dataset103_HnN", "Dataset104_HnN", "Dataset105_HnN", "Dataset111_HnN", "Dataset112_HnN", "Dataset113_HnN", "Dataset114_HnN", "Dataset115_HnN"]
dataset_ids_list_translate = ["Real", "DPM++ 2M 200", "DPM++ 2M Karras 200", "DPM++ 2M SDE 200", "DPM++ 2M SDE Karras 200", "Linear (1000) 200", "DPM++ 2M 1000", "DPM++ 2M Karras 1000", "DPM++ 2M SDE 1000", "DPM++ 2M SDE Karras 1000", "Linear (1000) 1000"]
results = []

for nnunet_id_idx, nnunet_id in enumerate(dataset_ids_list):
    nnunet_id_path = join(dsc_folder, nnunet_id)
    folder_name = nnunet_id
    sub_fold = 'hu_no_clipped'
    #for sub_fold in listdir(nnunet_id_path):
    json_file_path = join(nnunet_id_path, sub_fold, 'summary_instance.json')
    if not os.path.exists(json_file_path):
        result = {
            "Folder": dataset_ids_list_translate[nnunet_id_idx],
            "Sub_folder": sub_fold,
            "DSC type": 'instance',
            "Vertebrae": 'missing',
            "Discs": 'missing',
            "Spinal Canal": 'missing',
            "Spinal Cord": 'missing'
        }
        results.append(result)
    else:
        with open(json_file_path, 'r') as file:
            data = json.load(file)
        result = {
            "Folder": dataset_ids_list_translate[nnunet_id_idx],
            "Sub_folder": sub_fold,
            "DSC type": 'instance',
            "Vertebrae": data['Vertebrae'],
            "Discs": data['Discs'],
            "Spinal Canal": data['Spinal Canal'],
            "Spinal Cord": data['Spinal Cord']
        }
        results.append(result)

    json_file_path = join(nnunet_id_path, sub_fold, 'summary_semantic.json')
    if not os.path.exists(json_file_path):
        result = {
            "Folder": dataset_ids_list_translate[nnunet_id_idx],
            "Sub_folder": sub_fold,
            "DSC type": 'semantic',
            "Vertebrae": 'missing',
            "Discs": 'missing',
            "Spinal Canal": 'missing',
            "Spinal Cord": 'missing'
        }
        results.append(result)
    else:
        with open(json_file_path, 'r') as file:
            data = json.load(file)
        result = {
            "Folder": dataset_ids_list_translate[nnunet_id_idx],
            "Sub_folder": sub_fold,
            "DSC type": 'semantic',
            "Vertebrae": data['Vertebrae'],
            "Discs": data['Discs'],
            "Spinal Canal": data['Spinal Canal'],
            "Spinal Cord": data['Spinal Cord']
        }
        results.append(result)
print(results)
# Create a DataFrame from the results
df = pd.DataFrame(results)

# Save the DataFrame to an Excel file
df.to_excel('./metrics/DSC/resume_total_spine_seg.xlsx', index=False)

print("./metrics/DSC/resume_total_spine_seg.xlsx")

#### Total segmentator segmentation

In [ ]:
import multiprocessing
import os
from copy import deepcopy
from multiprocessing import Pool
import numpy as np
from nnunetv2.imageio.simpleitk_reader_writer import SimpleITKIO
from batchgenerators.utilities.file_and_folder_operations import subfiles, join, save_json, load_json, \
    isfile
from nnunetv2.utilities.json_export import recursive_fix_for_json_export
import sys
sys.path.append(".")
from test_utils import compute_metrics_on_folder

##### Real

In [ ]:
## General
test_folder = "../../nnUNet/nnUNet_test"
output_folder = "./metrics/DSC"
# Label specific
regions_or_labels = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]
folder_ref="../../nnUNet/nnUNet_raw/Dataset200_HnN/labelsTs"
# Experiment
dataset_id = "Dataset200_HnN"

for sub_folder in listdir(join(test_folder, dataset_id)):
    folder_pred=join(test_folder, dataset_id, sub_folder)
    output_file= join(output_folder, dataset_id, sub_folder, "summary.json")
    os.makedirs(join(output_folder, dataset_id, sub_folder), exist_ok=True)
    if os.path.exists(output_file):
        continue
    try:
        compute_metrics_on_folder(folder_ref=folder_ref, 
                                folder_pred=folder_pred, 
                                output_file=output_file,
                                image_reader_writer=SimpleITKIO(),
                                file_ending='.nii.gz',
                                regions_or_labels=regions_or_labels,
                                ignore_label=None,
                                num_processes=16,
                                chill=False)
        print(join(output_folder, dataset_id, sub_folder, "summary.json"))
    except:
        print(f"Problems with: {join(dataset_id, sub_folder, 'summary.json')}")

print("DONE")

##### Fake

In [ ]:
## General
test_folder = "../../nnUNet/nnUNet_test"
output_folder = "./metrics/DSC"
# Label specific
regions_or_labels = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]
folder_ref="../../nnUNet/nnUNet_raw/Dataset200_HnN/labelsTs"
# Experiment
dataset_ids_list = ["Dataset201_HnN", "Dataset202_HnN", "Dataset203_HnN", "Dataset204_HnN", "Dataset205_HnN", "Dataset211_HnN", "Dataset212_HnN", "Dataset213_HnN", "Dataset214_HnN", "Dataset215_HnN"]

for dataset_id in dataset_ids_list:
    for sub_folder in listdir(join(test_folder, dataset_id)):
        folder_pred=join(test_folder, dataset_id, sub_folder)
        output_file= join(output_folder, dataset_id, sub_folder, "summary.json")
        os.makedirs(join(output_folder, dataset_id, sub_folder), exist_ok=True)
        if os.path.exists(output_file):
            continue
        try:
            compute_metrics_on_folder(folder_ref=folder_ref, 
                                    folder_pred=folder_pred, 
                                    output_file=output_file,
                                    image_reader_writer=SimpleITKIO(),
                                    file_ending='.nii.gz',
                                    regions_or_labels=regions_or_labels,
                                    ignore_label=None,
                                    num_processes=16,
                                    chill=False)
            print(join(output_folder, dataset_id, sub_folder, "summary.json"))
        except:
            print(f"Problems with: {join(dataset_id, sub_folder, 'summary.json')}")

print("DONE")

##### Semantic segmentation evaluation

In [ ]:
import json
import numpy as np
import nibabel as nib
from os import listdir
from os.path import join
from multiprocessing import Pool, cpu_count

def get_semantic_seg_regions(pred_file_data, skull, claviculas, vertebrae, ribs, sternum, costal_cartilages, spinal_canal):
    return [
        np.isin(pred_file_data, skull).astype(np.uint8),
        np.isin(pred_file_data, claviculas).astype(np.uint8),
        np.isin(pred_file_data, vertebrae).astype(np.uint8),
        np.isin(pred_file_data, ribs).astype(np.uint8),
        np.isin(pred_file_data, sternum).astype(np.uint8),
        np.isin(pred_file_data, costal_cartilages).astype(np.uint8),
        np.isin(pred_file_data, spinal_canal).astype(np.uint8)
    ]

def get_TP_TN_FP_FN(ground_truth, prediction):
    TP = np.sum((ground_truth == 1) & (prediction == 1))
    TN = np.sum((ground_truth == 0) & (prediction == 0))
    FP = np.sum((ground_truth == 0) & (prediction == 1))
    FN = np.sum((ground_truth == 1) & (prediction == 0))
    return TP, TN, FP, FN

def compute_dsc(TP, FP, FN):
    return (2 * TP) / (2 * TP + FP + FN) if (2 * TP + FP + FN) > 0 else 0.0

def process_file(predictions_folder, pred_file_name, gt_folder, skull, claviculas, vertebrae, ribs, sternum, costal_cartilages, spinal_canal):
    gt_file_data = nib.load(join(gt_folder, pred_file_name)).get_fdata()
    pred_file_data = nib.load(join(predictions_folder, pred_file_name)).get_fdata()
    
    gt_regions = get_semantic_seg_regions(gt_file_data, skull, claviculas, vertebrae, ribs, sternum, costal_cartilages, spinal_canal)
    pred_regions = get_semantic_seg_regions(pred_file_data, skull, claviculas, vertebrae, ribs, sternum, costal_cartilages, spinal_canal)
    
    region_names = ["Skull", "Claviculas", "Vertebrae", "Ribs", "Sternum", "Costal Cartilages", "Spinal Canal"]
    
    file_results = {}
    for name, gt, pred in zip(region_names, gt_regions, pred_regions):
        TP, TN, FP, FN = get_TP_TN_FP_FN(gt, pred)
        file_results[name] = {"TP": TP, "TN": TN, "FP": FP, "FN": FN}
    
    return file_results

def compute_mean_dsc_folder_totalsegmentator(gt_folder, predictions_folder, output_file):
    skull = [17]
    claviculas = [14, 15]
    vertebrae = list(range(1, 14))
    ribs = list(range(18, 30))
    sternum = [30]
    costal_cartilages = [31]
    spinal_canal = [16]
    
    pred_files = [f for f in listdir(predictions_folder) if f.endswith('nii.gz')]
    
    with Pool(cpu_count()) as pool:
        results = pool.starmap(
            process_file,
            [(predictions_folder, f, gt_folder, skull, claviculas, vertebrae, ribs, sternum, costal_cartilages, spinal_canal) for f in pred_files]
        )
    
    aggregated_metrics = {name: {"TP": 0, "TN": 0, "FP": 0, "FN": 0} for name in ["Skull", "Claviculas", "Vertebrae", "Ribs", "Sternum", "Costal Cartilages", "Spinal Canal"]}
    
    for file_result in results:
        for region, metrics in file_result.items():
            for key in ["TP", "TN", "FP", "FN"]:
                aggregated_metrics[region][key] += metrics[key]
    
    dsc_results = {region: compute_dsc(metrics["TP"], metrics["FP"], metrics["FN"]) for region, metrics in aggregated_metrics.items()}
    
    with open(output_file, 'w') as f:
        json.dump(dsc_results, f, indent=4)
    
    print(f"Mean DSC results saved to {output_file}")


In [ ]:
import os
## General
test_folder = "../../nnUNet/nnUNet_test"
output_folder = "./metrics/DSC"
# Label specific
gt_folder="../../nnUNet/nnUNet_raw/Dataset200_HnN/labelsTs"
# Experiment
dataset_ids_list = ["Dataset200_HnN", "Dataset201_HnN", "Dataset202_HnN", "Dataset203_HnN", "Dataset204_HnN", "Dataset205_HnN", "Dataset211_HnN", "Dataset212_HnN", "Dataset213_HnN", "Dataset214_HnN", "Dataset215_HnN"]

for dataset_id in dataset_ids_list:
    for sub_folder in listdir(join(test_folder, dataset_id)):
        folder_pred=join(test_folder, dataset_id, sub_folder)
        output_file= join(output_folder, dataset_id, sub_folder, "summary_semantic.json")
        os.makedirs(join(output_folder, dataset_id, sub_folder), exist_ok=True)
        if os.path.exists(output_file):
            continue
        
        compute_mean_dsc_folder_totalsegmentator(gt_folder=gt_folder, predictions_folder=folder_pred, output_file=output_file)
        print(join(output_folder, dataset_id, sub_folder, "summary_semantic.json"))
        

print("DONE")

##### Instance segmentation evaluation

In [ ]:
# MEAN DSC
import json
import pandas as pd
from os import listdir
from os.path import join
epsilon=1e-6

problems = []
skull = [17]
claviculas = [14, 15]
vertebrae = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
ribs = [18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29]
sternum = [30]
costal_cartilages = [31]
spinal_canal = [16]

structure_name_list = ['Skull', 'Claviculas', 'Vertebrae', 'Ribs', 'Sternum', 'Costal Cartilages', 'Spinal Canal']

import os
## General
test_folder = "../../nnUNet/nnUNet_test"
output_folder = "./metrics/DSC"
# Label specific
gt_folder="../../nnUNet/nnUNet_raw/Dataset200_HnN/labelsTs"
# Experiment
dataset_ids_list = ["Dataset200_HnN", "Dataset201_HnN", "Dataset202_HnN", "Dataset203_HnN", "Dataset204_HnN", "Dataset205_HnN", "Dataset211_HnN", "Dataset212_HnN", "Dataset213_HnN", "Dataset214_HnN", "Dataset215_HnN"]

for dataset_id in dataset_ids_list:
    for sub_folder in listdir(join(test_folder, dataset_id)):
        folder_pred=join(test_folder, dataset_id, sub_folder)
        output_file= join(output_folder, dataset_id, sub_folder, "summary_instance.json")
        os.makedirs(join(output_folder, dataset_id, sub_folder), exist_ok=True)
        if os.path.exists(output_file):
            continue

        sumary_json_file = f"./metrics/DSC/{dataset_id}/{sub_folder}/summary.json"
        if not os.path.exists(sumary_json_file):
            problems.append(sumary_json_file)
            continue

        
        else:
            with open(sumary_json_file, 'r') as file:
                # Load the JSON data
                data = json.load(file)
                dsc_results = {}
                for idx_structure, structure in enumerate([vertebrae, discs, spinal_canal, spinal_cord]):
                    n_TP = 0
                    n_FP = 0
                    n_FN = 0
                    for case in data['metric_per_case']: # go to all predictions
                        for label_n in structure: # To all labels
                            if str(label_n) in case['metrics']:
                                n_TP += case['metrics'][str(label_n)]['TP']
                                n_FP += case['metrics'][str(label_n)]['FP']
                                n_FN += case['metrics'][str(label_n)]['FN']
                                
                    dsc = (2 * n_TP ) / (2 * n_TP + n_FP + n_FN )
                    dsc_results[structure_name_list[idx_structure]] = dsc
            
            with open(output_file, 'w') as f:
                json.dump(dsc_results, f, indent=4)
            
            print(f"Mean DSC results saved to {output_file}")
        
  
print(f"problems:")
for problem in problems:
    print(problem)



##### Make a excel sheet resuming

In [ ]:
dsc_folder = "./metrics/DSC"
dataset_ids_list = ["Dataset200_HnN", "Dataset201_HnN", "Dataset202_HnN", "Dataset203_HnN", "Dataset204_HnN", "Dataset205_HnN", "Dataset211_HnN", "Dataset212_HnN", "Dataset213_HnN", "Dataset214_HnN", "Dataset215_HnN"]
dataset_ids_list_translate = ["Real", "DPM++ 2M 200", "DPM++ 2M Karras 200", "DPM++ 2M SDE 200", "DPM++ 2M SDE Karras 200", "Linear (1000) 200", "DPM++ 2M 1000", "DPM++ 2M Karras 1000", "DPM++ 2M SDE 1000", "DPM++ 2M SDE Karras 1000", "Linear (1000) 1000"]
results = []

for nnunet_id_idx, nnunet_id in enumerate(dataset_ids_list):
    nnunet_id_path = join(dsc_folder, nnunet_id)
    folder_name = nnunet_id
    sub_fold = 'hu_no_clipped'
    #for sub_fold in listdir(nnunet_id_path):
    json_file_path = join(nnunet_id_path, sub_fold, 'summary_instance.json')
    if not os.path.exists(json_file_path):
        result = {
            "Folder": dataset_ids_list_translate[nnunet_id_idx],
            "Sub_folder": sub_fold,
            "DSC type": 'instance',
            "skull": 'missing',
            "claviculas": 'missing',
            "vertebrae": 'missing',
            "ribs": 'missing',
            "sternum": 'missing',
            "costal_cartilages": 'missing',
            "spinal_canal": 'missing'
        }
        results.append(result)

    else:
        with open(json_file_path, 'r') as file:
            data = json.load(file)
        result = {
            "Folder": dataset_ids_list_translate[nnunet_id_idx],
            "Sub_folder": sub_fold,
            "DSC type": 'instance',
            "skull": data['skull'],
            "claviculas": data['claviculas'],
            "vertebrae": data['vertebrae'],
            "ribs": data['ribs'],
            "sternum": data['sternum'],
            "costal_cartilages": data['costal_cartilages'],
            "spinal_canal": data['spinal_canal']
        }
        results.append(result)

    json_file_path = join(nnunet_id_path, sub_fold, 'summary_semantic.json')
    if not os.path.exists(json_file_path):
        result = {
            "Folder": dataset_ids_list_translate[nnunet_id_idx],
            "Sub_folder": sub_fold,
            "DSC type": 'semantic',
            "skull": 'missing',
            "claviculas": 'missing',
            "vertebrae": 'missing',
            "ribs": 'missing',
            "sternum": 'missing',
            "costal_cartilages": 'missing',
            "spinal_canal": 'missing'
        }
        results.append(result)
    else:
        with open(json_file_path, 'r') as file:
            data = json.load(file)
        result = {
            "Folder": dataset_ids_list_translate[nnunet_id_idx],
            "Sub_folder": sub_fold,
            "DSC type": 'semantic',
            "skull": data['Skull'],
            "claviculas": data['Claviculas'],
            "vertebrae": data['Vertebrae'],
            "ribs": data['Ribs'],
            "sternum": data['Sternum'],
            "costal_cartilages": data['Costal Cartilages'],
            "spinal_canal": data['Spinal Canal']
        }
        results.append(result)
print(results)
# Create a DataFrame from the results
df = pd.DataFrame(results)

# Save the DataFrame to an Excel file
df.to_excel('./metrics/DSC/resume_totalsegmentator.xlsx', index=False)

print("./metrics/DSC/resume_totalsegmentator.xlsx")

#### AMASS segmentation (Maxilla Mandibula)

In [ ]:
import multiprocessing
import os
from copy import deepcopy
from multiprocessing import Pool
import numpy as np
from nnunetv2.imageio.simpleitk_reader_writer import SimpleITKIO
from batchgenerators.utilities.file_and_folder_operations import subfiles, join, save_json, load_json, \
    isfile
from nnunetv2.utilities.json_export import recursive_fix_for_json_export
import sys
sys.path.append(".")
from test_utils import compute_metrics_on_folder

##### Real

In [ ]:
## General
test_folder = "../../nnUNet/nnUNet_test"
output_folder = "./metrics/DSC"
# Label specific
regions_or_labels = [1, 2]
folder_ref="../../nnUNet/nnUNet_raw/Dataset300_HnN/labelsTs"
# Experiment
dataset_id = "Dataset300_HnN"

for sub_folder in listdir(join(test_folder, dataset_id)):
    folder_pred=join(test_folder, dataset_id, sub_folder)
    output_file= join(output_folder, dataset_id, sub_folder, "summary.json")
    os.makedirs(join(output_folder, dataset_id, sub_folder), exist_ok=True)
    if os.path.exists(output_file):
        continue
    try:
        compute_metrics_on_folder(folder_ref=folder_ref, 
                                folder_pred=folder_pred, 
                                output_file=output_file,
                                image_reader_writer=SimpleITKIO(),
                                file_ending='.nii.gz',
                                regions_or_labels=regions_or_labels,
                                ignore_label=None,
                                num_processes=16,
                                chill=False)
        print(join(output_folder, dataset_id, sub_folder, "summary.json"))
    except:
        print(f"Problems with: {join(dataset_id, sub_folder, 'summary.json')}")

print("DONE")

##### Fake

In [ ]:
## General
test_folder = "../../nnUNet/nnUNet_test"
output_folder = "./metrics/DSC"
# Label specific
regions_or_labels = [1, 2]
folder_ref="../../nnUNet/nnUNet_raw/Dataset300_HnN/labelsTs"
# Experiment
dataset_ids_list = ["Dataset311_HnN", "Dataset312_HnN", "Dataset313_HnN", "Dataset314_HnN", "Dataset315_HnN"]

for dataset_id in dataset_ids_list:
    for sub_folder in listdir(join(test_folder, dataset_id)):
        folder_pred=join(test_folder, dataset_id, sub_folder)
        output_file= join(output_folder, dataset_id, sub_folder, "summary.json")
        os.makedirs(join(output_folder, dataset_id, sub_folder), exist_ok=True)
        if os.path.exists(output_file):
            continue
        try:
            compute_metrics_on_folder(folder_ref=folder_ref, 
                                    folder_pred=folder_pred, 
                                    output_file=output_file,
                                    image_reader_writer=SimpleITKIO(),
                                    file_ending='.nii.gz',
                                    regions_or_labels=regions_or_labels,
                                    ignore_label=None,
                                    num_processes=16,
                                    chill=False)
            print(join(output_folder, dataset_id, sub_folder, "summary.json"))
        except:
            print(f"Problems with: {join(dataset_id, sub_folder, 'summary.json')}")

print("DONE")

##### Semantic segmentation evaluation

In [ ]:
import json
import numpy as np
import nibabel as nib
from os import listdir
from os.path import join
from multiprocessing import Pool, cpu_count

def get_semantic_seg_regions(pred_file_data, mandibula, maxilla):
    return [
        np.isin(pred_file_data, mandibula).astype(np.uint8),
        np.isin(pred_file_data, maxilla).astype(np.uint8)
    ]

def get_TP_TN_FP_FN(ground_truth, prediction):
    TP = np.sum((ground_truth == 1) & (prediction == 1))
    TN = np.sum((ground_truth == 0) & (prediction == 0))
    FP = np.sum((ground_truth == 0) & (prediction == 1))
    FN = np.sum((ground_truth == 1) & (prediction == 0))
    return TP, TN, FP, FN

def compute_dsc(TP, FP, FN):
    return (2 * TP) / (2 * TP + FP + FN) if (2 * TP + FP + FN) > 0 else 0.0

def process_file(predictions_folder, pred_file_name, gt_folder, mandibula, maxilla):
    gt_file_data = nib.load(join(gt_folder, pred_file_name)).get_fdata()
    pred_file_data = nib.load(join(predictions_folder, pred_file_name)).get_fdata()
    
    gt_regions = get_semantic_seg_regions(gt_file_data, mandibula, maxilla)
    pred_regions = get_semantic_seg_regions(pred_file_data, mandibula, maxilla)
    
    region_names = ["Mandibula", "Maxilla"]
    
    file_results = {}
    for name, gt, pred in zip(region_names, gt_regions, pred_regions):
        TP, TN, FP, FN = get_TP_TN_FP_FN(gt, pred)
        file_results[name] = {"TP": TP, "TN": TN, "FP": FP, "FN": FN}
    
    return file_results

def compute_mean_dsc_folder_amasss(gt_folder, predictions_folder, output_file):
    mandibula = [1]
    maxilla = [2]
    
    pred_files = [f for f in listdir(predictions_folder) if f.endswith('nii.gz')]
    
    with Pool(cpu_count()) as pool:
        results = pool.starmap(
            process_file,
            [(predictions_folder, f, gt_folder, mandibula, maxilla) for f in pred_files]
        )
    
    aggregated_metrics = {name: {"TP": 0, "TN": 0, "FP": 0, "FN": 0} for name in ["Mandibula", "Maxilla"]}
    
    for file_result in results:
        for region, metrics in file_result.items():
            for key in ["TP", "TN", "FP", "FN"]:
                aggregated_metrics[region][key] += metrics[key]
    
    dsc_results = {region: compute_dsc(metrics["TP"], metrics["FP"], metrics["FN"]) for region, metrics in aggregated_metrics.items()}
    
    with open(output_file, 'w') as f:
        json.dump(dsc_results, f, indent=4)
    
    print(f"Mean DSC results saved to {output_file}")


In [ ]:
import os
## General
test_folder = "../../nnUNet/nnUNet_test"
output_folder = "./metrics/DSC"
# Label specific
gt_folder="../../nnUNet/nnUNet_raw/Dataset300_HnN/labelsTs"
# Experiment
dataset_ids_list = ["Dataset300_HnN", "Dataset311_HnN", "Dataset312_HnN", "Dataset313_HnN", "Dataset314_HnN", "Dataset315_HnN"]

for dataset_id in dataset_ids_list:
    for sub_folder in listdir(join(test_folder, dataset_id)):
        folder_pred=join(test_folder, dataset_id, sub_folder)
        output_file= join(output_folder, dataset_id, sub_folder, "summary_semantic.json")
        os.makedirs(join(output_folder, dataset_id, sub_folder), exist_ok=True)
        if os.path.exists(output_file):
            continue
        
        compute_mean_dsc_folder_amasss(gt_folder=gt_folder, predictions_folder=folder_pred, output_file=output_file)
        print(join(output_folder, dataset_id, sub_folder, "summary_semantic.json"))
        

print("DONE")

##### Instance segmentation evaluation

In [ ]:
# MEAN DSC
import json
import pandas as pd
from os import listdir
from os.path import join
epsilon=1e-6

mandibula = [1]
maxilla = [2]
structure_name_list = ['mandibula', 'maxilla']

import os
## General
test_folder = "../../nnUNet/nnUNet_test"
output_folder = "./metrics/DSC"
# Label specific
gt_folder="../../nnUNet/nnUNet_raw/Dataset300_HnN/labelsTs"
# Experiment
dataset_ids_list = ["Dataset300_HnN", "Dataset311_HnN", "Dataset312_HnN", "Dataset313_HnN", "Dataset314_HnN", "Dataset315_HnN"]

problems = []
for dataset_id in dataset_ids_list:
    for sub_folder in listdir(join(test_folder, dataset_id)):
        folder_pred=join(test_folder, dataset_id, sub_folder)
        output_file= join(output_folder, dataset_id, sub_folder, "summary_instance.json")
        os.makedirs(join(output_folder, dataset_id, sub_folder), exist_ok=True)
        if os.path.exists(output_file):
            continue

        sumary_json_file = f"./metrics/DSC/{dataset_id}/{sub_folder}/summary.json"
        if not os.path.exists(sumary_json_file):
            problems.append(sumary_json_file)
            continue

        
        else:
            with open(sumary_json_file, 'r') as file:
                # Load the JSON data
                data = json.load(file)
                dsc_results = {}
                for idx_structure, structure in enumerate([mandibula, maxilla]):
                    n_TP = 0
                    n_FP = 0
                    n_FN = 0
                    for case in data['metric_per_case']: # go to all predictions
                        for label_n in structure: # To all labels
                            if str(label_n) in case['metrics']:
                                n_TP += case['metrics'][str(label_n)]['TP']
                                n_FP += case['metrics'][str(label_n)]['FP']
                                n_FN += case['metrics'][str(label_n)]['FN']
                                
                    dsc = (2 * n_TP ) / (2 * n_TP + n_FP + n_FN )
                    dsc_results[structure_name_list[idx_structure]] = dsc
            
            with open(output_file, 'w') as f:
                json.dump(dsc_results, f, indent=4)
            
            print(f"Mean DSC results saved to {output_file}")
        
  
print(f"problems:")
for problem in problems:
    print(problem)





##### Make a excel sheet resuming

In [ ]:
dsc_folder = "./metrics/DSC"
dataset_ids_list = ["Dataset300_HnN", "Dataset311_HnN", "Dataset312_HnN", "Dataset313_HnN", "Dataset314_HnN", "Dataset315_HnN"]
dataset_ids_list_translate = ["Real", "DPM++ 2M 1000", "DPM++ 2M Karras 1000", "DPM++ 2M SDE 1000", "DPM++ 2M SDE Karras 1000", "Linear (1000) 1000"]
results = []

for nnunet_id_idx, nnunet_id in enumerate(dataset_ids_list):
    nnunet_id_path = join(dsc_folder, nnunet_id)
    folder_name = nnunet_id
    sub_fold = 'hu_no_clipped'
    #for sub_fold in listdir(nnunet_id_path):
    json_file_path = join(nnunet_id_path, sub_fold, 'summary_instance.json')
    if not os.path.exists(json_file_path):
        result = {
            "Folder": dataset_ids_list_translate[nnunet_id_idx],
            "Sub_folder": sub_fold,
            "DSC type": 'instance',
            "mandibula": 'missing',
            "maxilla": 'missing'
        }
        results.append(result)

    else:
        with open(json_file_path, 'r') as file:
            data = json.load(file)
        result = {
            "Folder": dataset_ids_list_translate[nnunet_id_idx],
            "Sub_folder": sub_fold,
            "DSC type": 'instance',
            "mandibula": data['mandibula'],
            "maxilla": data['maxilla']
        }
        results.append(result)

    json_file_path = join(nnunet_id_path, sub_fold, 'summary_semantic.json')
    if not os.path.exists(json_file_path):
        result = {
            "Folder": dataset_ids_list_translate[nnunet_id_idx],
            "Sub_folder": sub_fold,
            "DSC type": 'semantic',
            "mandibula": 'missing',
            "maxilla": 'missing'
        }
        results.append(result)
    else:
        with open(json_file_path, 'r') as file:
            data = json.load(file)
        result = {
            "Folder": dataset_ids_list_translate[nnunet_id_idx],
            "Sub_folder": sub_fold,
            "DSC type": 'semantic',
            "mandibula": data['Mandibula'],
            "maxilla": data['Maxilla']
        }
        results.append(result)
print(results)
# Create a DataFrame from the results
df = pd.DataFrame(results)

# Save the DataFrame to an Excel file
df.to_excel('./metrics/DSC/resume_amasss.xlsx', index=False)

print("./metrics/DSC/resume_amasss.xlsx")

### Tumour segmentation

#### Head and neck tumour segmentation

In [ ]:
import multiprocessing
import os
import shutil
from os import listdir
from os.path import join
from copy import deepcopy
from multiprocessing import Pool
import numpy as np
from nnunetv2.imageio.simpleitk_reader_writer import SimpleITKIO
from batchgenerators.utilities.file_and_folder_operations import subfiles, join, save_json, load_json, \
    isfile
from nnunetv2.utilities.json_export import recursive_fix_for_json_export
import sys
sys.path.append(".")
from test_utils import compute_metrics_on_folder
import pandas as pd
import json

In [ ]:
metrics_root_path = "./metrics/DSC"
test_folder = "../../nnUNet/nnUNet_test/"
regions_or_labels = [1]
folder_ref = "../../nnUNet/nnUNet_raw/Dataset910_HnN/labelsTs"
# The dataset_name_list is the one to change.
dataset_name_list_200 =['Dataset500_HnN', 'Dataset501_HnN', 'Dataset502_HnN', 'Dataset503_HnN', 'Dataset504_HnN', 
                        'Dataset510_HnN', 'Dataset511_HnN', 'Dataset512_HnN', 'Dataset513_HnN', 'Dataset514_HnN', 
                        'Dataset520_HnN', 'Dataset521_HnN', 'Dataset522_HnN', 'Dataset523_HnN', 'Dataset524_HnN', 
                        'Dataset530_HnN', 'Dataset531_HnN', 'Dataset532_HnN', 'Dataset533_HnN', 'Dataset534_HnN', 
                        'Dataset550_HnN', 'Dataset551_HnN', 'Dataset552_HnN', 'Dataset553_HnN', 'Dataset554_HnN', 
                        'Dataset560_HnN', 'Dataset561_HnN', 'Dataset562_HnN', 'Dataset563_HnN', 'Dataset564_HnN', 'Dataset910_HnN', 'Dataset982_HnN_GAN']

dataset_name_list_1000 = ['Dataset540_HnN', 'Dataset541_HnN', 'Dataset542_HnN', 'Dataset543_HnN', 'Dataset544_HnN',
                          'Dataset570_HnN', 'Dataset571_HnN', 'Dataset572_HnN', 'Dataset573_HnN', 'Dataset574_HnN', 
                          'Dataset580_HnN', 'Dataset581_HnN', 'Dataset582_HnN', 'Dataset583_HnN', 'Dataset584_HnN']

dataset_name_list_200_last = [
    'Dataset555_HnN', 'Dataset556_HnN', 'Dataset557_HnN', 'Dataset558_HnN', 'Dataset559_HnN',
    'Dataset565_HnN', 'Dataset566_HnN', 'Dataset567_HnN', 'Dataset568_HnN', 'Dataset569_HnN',
    'Dataset590_HnN', 'Dataset591_HnN', 'Dataset592_HnN', 'Dataset593_HnN', 'Dataset594_HnN',
    'Dataset595_HnN', 'Dataset596_HnN', 'Dataset597_HnN', 'Dataset598_HnN', 'Dataset599_HnN',
    'Dataset983_HnN_GAN'
]



In [ ]:
test_folder = "../../nnUNet/nnUNet_test"
for dataset_id in dataset_name_list_1000:
    dataset_id_path = join(test_folder, dataset_id)
    for sub_folder in listdir(dataset_id_path):
        complete_path = join(dataset_id_path, sub_folder)
        if sub_folder == "nnUNetTrainer__nnUNetPlans__3d_fullres":
            #shutil.rmtree(complete_path)
            print(f"Removed: {complete_path}")
            continue
        if len(listdir(complete_path))==198:
            continue
        else:
            print(f"complete_path: {complete_path}")


In [ ]:
for dataset_name in dataset_name_list_200:
    for sub_folder in listdir(join(test_folder, dataset_name)):
        output_folder = join(metrics_root_path, dataset_name, sub_folder)
        output_file = join(output_folder, "summary.json")
        os.makedirs(output_folder, exist_ok=True)

        folder_pred = join(test_folder, dataset_name, sub_folder)

        if os.path.exists(output_file):
            continue
        else:
            try:
                compute_metrics_on_folder(folder_ref=folder_ref, 
                                        folder_pred=folder_pred, 
                                        output_file=output_file,
                                        image_reader_writer=SimpleITKIO(),
                                        file_ending='.nii.gz',
                                        regions_or_labels=regions_or_labels,
                                        ignore_label=None,
                                        num_processes=8,
                                        chill=False)
            except:
                print(f"Problems folder_pred: {folder_pred}")
print(f"DONE")





In [ ]:
for dataset_name in dataset_name_list_1000:
    for sub_folder in listdir(join(test_folder, dataset_name)):
        output_folder = join(metrics_root_path, dataset_name, sub_folder)
        output_file = join(output_folder, "summary.json")
        os.makedirs(output_folder, exist_ok=True)

        folder_pred = join(test_folder, dataset_name, sub_folder)

        if os.path.exists(output_file):
            continue
        else:
            try:
                compute_metrics_on_folder(folder_ref=folder_ref, 
                                        folder_pred=folder_pred, 
                                        output_file=output_file,
                                        image_reader_writer=SimpleITKIO(),
                                        file_ending='.nii.gz',
                                        regions_or_labels=regions_or_labels,
                                        ignore_label=None,
                                        num_processes=8,
                                        chill=False)
            except:
                print(f"Problems folder_pred: {folder_pred}")
print(f"DONE")

In [ ]:
for dataset_name in dataset_name_list_200_last:
    for sub_folder in listdir(join(test_folder, dataset_name)):
        output_folder = join(metrics_root_path, dataset_name, sub_folder)
        output_file = join(output_folder, "summary.json")
        os.makedirs(output_folder, exist_ok=True)

        folder_pred = join(test_folder, dataset_name, sub_folder)

        if os.path.exists(output_file):
            continue
        else:
            try:
                print(f"Doing {folder_pred}")
                compute_metrics_on_folder(folder_ref=folder_ref, 
                                        folder_pred=folder_pred, 
                                        output_file=output_file,
                                        image_reader_writer=SimpleITKIO(),
                                        file_ending='.nii.gz',
                                        regions_or_labels=regions_or_labels,
                                        ignore_label=None,
                                        num_processes=8,
                                        chill=False)
            except:
                print(f"Problems folder_pred: {folder_pred}")
print(f"DONE")


In [ ]:
##### Compress all info in a excel file
data_list = []

dataset_name_list_total =['Dataset500_HnN', 'Dataset501_HnN', 'Dataset502_HnN', 'Dataset503_HnN', 'Dataset504_HnN', 
                        'Dataset510_HnN', 'Dataset511_HnN', 'Dataset512_HnN', 'Dataset513_HnN', 'Dataset514_HnN', 
                        'Dataset520_HnN', 'Dataset521_HnN', 'Dataset522_HnN', 'Dataset523_HnN', 'Dataset524_HnN', 
                        'Dataset530_HnN', 'Dataset531_HnN', 'Dataset532_HnN', 'Dataset533_HnN', 'Dataset534_HnN', 
                        'Dataset550_HnN', 'Dataset551_HnN', 'Dataset552_HnN', 'Dataset553_HnN', 'Dataset554_HnN', 
                        'Dataset560_HnN', 'Dataset561_HnN', 'Dataset562_HnN', 'Dataset563_HnN', 'Dataset564_HnN',
                        'Dataset540_HnN', 'Dataset541_HnN', 'Dataset542_HnN', 'Dataset543_HnN', 'Dataset544_HnN',
                        'Dataset570_HnN', 'Dataset571_HnN', 'Dataset572_HnN', 'Dataset573_HnN', 'Dataset574_HnN', 
                        'Dataset580_HnN', 'Dataset581_HnN', 'Dataset582_HnN', 'Dataset583_HnN', 'Dataset584_HnN', 
                        'Dataset910_HnN', 'Dataset982_HnN']

for dataset_name in dataset_name_list_total:
    for sub_folder in listdir(join(metrics_root_path, dataset_name)):
        output_folder = join(metrics_root_path, dataset_name, sub_folder)
        output_file = join(output_folder, "summary.json")
        if sub_folder!="hu_no_clipped":
            continue
        if not os.path.exists(output_file): 
            data_list.append([f"{dataset_name}_{sub_folder}", "Missing", 'Missing'])
            continue
        else:
            with open(output_file, "r") as file:
                results = json.load(file)
            Dice = results['foreground_mean']['Dice']
            IoU = results['foreground_mean']['IoU']
            #print(f"{dataset_name}_{sub_folder}", round(Dice['mean'], 4), round(IoU['mean'], 4))
            data_list.append([f"{dataset_name}_{sub_folder}", round(Dice['mean'], 4), round(IoU['mean'], 4)])

header = ["Dataset", "DSC", "IoU"]
df = pd.DataFrame(data_list, columns=header)
output_file_excel = "./metrics/DSC/summary_dsc.xlsx"
df.to_excel(output_file_excel, index=False)
print(output_file_excel)

In [ ]:
#### Finding best segmented case

best_seg_dsc = 0

best_cases_names = []
for dataset_name in dataset_name_list_200+dataset_name_list_1000:
    for sub_folder in listdir(join(metrics_root_path, dataset_name)):
        output_folder = join(metrics_root_path, dataset_name, sub_folder)
        output_file = join(output_folder, "summary.json")
        if not os.path.exists(output_file): 
            continue
        else:
            with open(output_file, "r") as file:
                results = json.load(file)
            for case in results['metric_per_case']:
                Dice = case['metrics']['1']['Dice']
                if Dice>=0.5:
                    best_seg_case = f"{dataset_name}_{sub_folder}_{case['prediction_file'].split('/')[-1]}: {Dice}"
                    print(f"best_seg_case: {best_seg_case}")
                    print(case['prediction_file'])
                    best_cases_names.append(f"{dataset_name}_{sub_folder}")
                    print("############")
                if Dice > best_seg_dsc:
                    best_seg_dsc = Dice
                    best_seg_case = f"{dataset_name}_{sub_folder}_{case['prediction_file'].split('/')[-1]}: {Dice}"
                    #print(f"best_seg_case: {best_seg_case}")
                    #print(case['prediction_file'])
                    #print("############")
            

print(best_cases_names)

In [ ]:
from collections import Counter
# Count the occurrences of each element
element_counts = Counter(best_cases_names)
element_counts


#### Brain tumour segmentation

In [ ]:
import multiprocessing
import os
from os import listdir
from os.path import join
from copy import deepcopy
from multiprocessing import Pool
import numpy as np
from nnunetv2.imageio.simpleitk_reader_writer import SimpleITKIO
from batchgenerators.utilities.file_and_folder_operations import subfiles, join, save_json, load_json, \
    isfile
from nnunetv2.utilities.json_export import recursive_fix_for_json_export
import sys
sys.path.append(".")
from test_utils import compute_metrics_on_folder

##### REAL

In [ ]:
## General
test_folder = "../../nnUNet/nnUNet_test"
output_folder = "./metrics/DSC"
# Label specific
regions_or_labels = [(1,2,3), (2,3), (3)]
folder_ref="../../nnUNet/nnUNet_raw/Dataset960_BraTS/labelsTs"
# Experiment
dataset_id = "Dataset960_BraTS"

for sub_folder in listdir(join(test_folder, dataset_id)):
    folder_pred=join(test_folder, dataset_id, sub_folder)
    print(f"folder_pred: {folder_pred}")
    output_file= join(output_folder, dataset_id, sub_folder, "summary.json")
    os.makedirs(join(output_folder, dataset_id, sub_folder), exist_ok=True)
    if os.path.exists(output_file):
        continue
    #try:
    else:
        print(f"output_file: {output_file}")
        compute_metrics_on_folder(folder_ref=folder_ref, 
                                folder_pred=folder_pred, 
                                output_file=output_file,
                                image_reader_writer=SimpleITKIO(),
                                file_ending='.nii.gz',
                                regions_or_labels=regions_or_labels,
                                ignore_label=None,
                                num_processes=8,
                                chill=True)
        print(join(output_folder, dataset_id, sub_folder, "summary.json"))
    #except:
    #    print(f"Problems with: {join(dataset_id, sub_folder, 'summary.json')}")

print("DONE")

##### FAKE

In [ ]:
## General
test_folder = "../../nnUNet/nnUNet_test"
output_folder = "./metrics/DSC"
# Label specific
regions_or_labels = [(1,2,3), (2,3), (3)]
folder_ref="../../nnUNet/nnUNet_raw/Dataset960_BraTS/labelsTs"
# Experiment
dataset_ids_list = ["Dataset996_BraTS_GAN" ,"Dataset600_BraTS", "Dataset601_BraTS", "Dataset602_BraTS", "Dataset603_BraTS", "Dataset604_BraTS", "Dataset610_BraTS", "Dataset611_BraTS", "Dataset612_BraTS", "Dataset613_BraTS", "Dataset614_BraTS", "Dataset620_BraTS", "Dataset621_BraTS", "Dataset622_BraTS", "Dataset623_BraTS", "Dataset624_BraTS"]

for dataset_id in dataset_ids_list:
    for sub_folder in listdir(join(test_folder, dataset_id)):
        folder_pred=join(test_folder, dataset_id, sub_folder)
        output_file= join(output_folder, dataset_id, sub_folder, "summary.json")
        os.makedirs(join(output_folder, dataset_id, sub_folder), exist_ok=True)
        if os.path.exists(output_file):
            continue
        try:
            print(f"output_file: {output_file}")
            compute_metrics_on_folder(folder_ref=folder_ref, 
                                    folder_pred=folder_pred, 
                                    output_file=output_file,
                                    image_reader_writer=SimpleITKIO(),
                                    file_ending='.nii.gz',
                                    regions_or_labels=regions_or_labels,
                                    ignore_label=None,
                                    num_processes=8,
                                    chill=False)
            print(join(output_folder, dataset_id, sub_folder, "summary.json"))
        except:
            print(f"Problems with: {join(dataset_id, sub_folder, 'summary.json')}")

print("DONE")

In [ ]:
##### Compress all info in a excel file
data_list = []

for dataset_name in ["Dataset960_BraTS"]+dataset_ids_list:
    print(f"dataset_name: {dataset_name}")
    for sub_folder in listdir(join(metrics_root_path, dataset_name)):
        output_folder = join(metrics_root_path, dataset_name, sub_folder)
        output_file = join(output_folder, "summary.json")
        if sub_folder!="no_clipped":
            continue
        if not os.path.exists(output_file): 
            data_list.append([f"{dataset_name}_{sub_folder}", "Missing", 'Missing'])
            continue
        else:
            with open(output_file, "r") as file:
                results = json.load(file)
            Dice_mean = results['foreground_mean']['Dice']['mean']
            Dice_std = results['foreground_mean']['Dice']['std']
            Dice = results['mean']
            wt_mean = results['mean']['(1, 2, 3)']['Dice']['mean']
            wt_std = results['mean']['(1, 2, 3)']['Dice']['std']
            tc_mean = results['mean']['(2, 3)']['Dice']['mean']
            tc_std = results['mean']['(2, 3)']['Dice']['std']
            et_mean = results['mean']['3']['Dice']['mean']
            et_std = results['mean']['3']['Dice']['std']
            #IoU = results['mean']['IoU']
            #print(f"{dataset_name}_{sub_folder}", round(Dice['mean'], 4), round(IoU['mean'], 4))
            data_list.append([f"{dataset_name}_{sub_folder}", 
                              f"{round(wt_mean, 4)} ±{round(wt_std, 4)}",
                              f"{round(tc_mean, 4)} ±{round(tc_std, 4)}",
                              f"{round(et_mean, 4)} ±{round(et_std, 4)}",
                              f"{round(Dice_mean, 4)} ±{round(Dice_std, 4)}"
                              ]
                              )

header = ["Dataset", "wt", "tc", "et", 'mean']
df = pd.DataFrame(data_list, columns=header)
output_file_excel = "./metrics/DSC/summary_dsc_brats.xlsx"
df.to_excel(output_file_excel, index=False)
print(output_file_excel)

---
## MS-SSIM computation

In [ ]:
import argparse
import os
import numpy as np
import torch
import sys
import json
from glob import glob
from generative.metrics import MultiScaleSSIMMetric
from tqdm import tqdm
sys.path.append(".")
from test_utils import get_data_loader, get_BraTS_data_loader

In [ ]:
# Test multiprocessing
import multiprocessing
image_key = 'image'
clip_min = -200 # TODO -> this is changed for the cases with -1000 and 1000
clip_max = 200
data_dir = "../../HnN_cancer_data/HnN_cancer_data_1_1_1_256_256_256/data"

image_files = sorted(glob(os.path.join(data_dir, "*.nii.gz")))  
data_list = [{"image": img} for img in image_files]

dataloader_1, dataloader_2 = get_data_loader(image_key, clip_min, clip_max, data_list[:5], pad_or_crop=True, two_loaders=True, cache_rate=[0,0], for_ms_ssim=True, not_normalise=True)

device = torch.device("cpu")
ms_ssim = MultiScaleSSIMMetric(spatial_dims=3, data_range=1.0, kernel_size=7)

print("Computing MS-SSIM (this takes a while)...")

def compute_ms_ssim(args):
    """Compute MS-SSIM between two images."""
    print("Computing MSSSIM")
    img1, img2, device = args
    ms_ssim_value = ms_ssim(img1.to(device), img2.to(device)).item()
    print(f"ms_ssim_value: {ms_ssim_value}")
    return ms_ssim_value

def main(dataloader_1, dataloader_2, device):
    ms_ssim_list = []
    for step_1, batch in enumerate(dataloader_1):
        print(f"Doing case {step_1}")
        img = batch['image']
        print(f"img.max: {img.max()}")
        print(f"img.min: {img.min()}")

        # Prepare arguments for multiprocessing
        args = []
        pbar2 = tqdm(enumerate(dataloader_2), total=len(dataloader_2))
        for step_2, batch2 in pbar2:
            if step_1 == step_2:
                continue
            img2 = batch2['image']
            args.append((img, img2, device))
            if step_2==10:
                break
        print(f"args: {len(args)}")
        # Use multiprocessing to compute MS-SSIM with 8 CPUs
        with multiprocessing.Pool(processes=1) as pool:  # Limit to 8 CPUs
            results = list(pool.imap(compute_ms_ssim, args))
        
        ms_ssim_list.extend(results)
        break
    return ms_ssim_list

ms_ssim_list = main(dataloader_1, dataloader_2, device)
ms_ssim_list


### Fake

In [ ]:
image_key = 'image'
clip_min = -200 # TODO -> this is changed for the cases with -1000 and 1000
clip_max = 200
data_dir = "../results/Synthetic_Datasets/Whole_scans/Bone_segmentation/200/DPM++_2M"

image_files = sorted(glob(os.path.join(data_dir, "*.nii.gz")))  
data_list = [{"image": img} for img in image_files]

dataloader_1, dataloader_2 = get_data_loader(image_key, clip_min, clip_max, data_list, pad_or_crop=True, two_loaders=True, cache_rate=[0,1])

device = torch.device("cpu")
ms_ssim = MultiScaleSSIMMetric(spatial_dims=3, data_range=1.0, kernel_size=7)

print("Computing MS-SSIM (this takes a while)...")
ms_ssim_list = []

for step_1, batch in enumerate(dataloader_1):
    pbar2 = tqdm(enumerate(dataloader_2), total=len(dataloader_2))
    print(f"Doing case {step_1}")
    img = batch['image']
    for step_2, batch2 in pbar2:
        img2 = batch2['image']
        if step_1 == step_2:
            continue
        ms_ssim_value = ms_ssim(img.to(device), img2.to(device)).item()
        ms_ssim_list.append(ms_ssim_value)
        pbar2.set_postfix({"ms_ssim": float(ms_ssim_value)})

ms_ssim_list = np.array(ms_ssim_list)
print("Calculated MS-SSIMs. Computing mean ...")
print(f"Mean MS-SSIM: {ms_ssim_list.mean():.6f}")

mm_ssim = {}
mm_ssim["mean"] = float(ms_ssim_list.mean())
#with open("./metrics/MS-SSIM/fake_mm_ssim.json", "w") as json_file:
#    json.dump(mm_ssim, json_file, indent=4)

### Real

In [ ]:
image_key = 'image'
clip_min = -200 # TODO -> this is changed for the cases with -1000 and 1000
clip_max = 200
data_dir = "../../HnN_cancer_data/HnN_cancer_data_1_1_1_256_256_256"

with open("../../HnN_cancer_data/HnN_cancer_data_1_1_1_256_256_256/data_split.json", 'r') as f:
    data_split = json.load(f)
    training = data_split['training']
image_files = []
for file_name in training:
    image_files.append(file_name['image'])
data_list = [{"image": os.path.join(data_dir, img)} for img in image_files]
dataloader_1, dataloader_2 = get_data_loader(image_key, clip_min, clip_max, data_list, two_loaders=True, cache_rate=[0,1])

device = torch.device("cuda")
ms_ssim = MultiScaleSSIMMetric(spatial_dims=3, data_range=1.0, kernel_size=7)

print("Computing MS-SSIM (this takes a while)...")
ms_ssim_list = []
pbar = tqdm(enumerate(dataloader_1), total=len(dataloader_1))
for step_1, batch in pbar:
    img = batch['image']
    for step_2, batch2 in enumerate(dataloader_2):
        img2 = batch2['image']
        if step_1 == step_2:
            continue
        ms_ssim_list.append(ms_ssim(img.to(device), img2.to(device)).item())
    pbar.update()

ms_ssim_list = np.array(ms_ssim_list)
print("Calculated MS-SSIMs. Computing mean ...")
print(f"Mean MS-SSIM: {ms_ssim_list.mean():.6f}")

mm_ssim = {}
for idx, result in enumerate(ms_ssim_list):
    mm_ssim[image_files[idx].split('/')[-1].split('.nii.gz')[0]] = result
mm_ssim["mean"] = ms_ssim_list.mean():.6f
with open("./metrics/MS-SSIM/real_mm_ssim.json", "w") as json_file:
    json.dump(mm_ssim, json_file, indent=4)

In [ ]:
# Create a excel file
with open("./metrics/MS-SSIM/real_mm_ssim.json", "r") as json_file:
    data_real = json.load(json_file)
with open("./metrics/MS-SSIM/fake_mm_ssim.json", "r") as json_file:
    data_fake = json.load(json_file)
data_list = []
for key_case in data_real:
    data_list.append([key_case, data_real[key_case], data_fake[key_case]])
header = ["Case", "MS-SSIM real", "MS-SSIM fake"]
df = pd.DataFrame(data_list, columns=header)
output_file = "./metrics/MS-SSIM/summary.xlsx"
df.to_excel(output_file, index=False)

---
## MAE - comparision between real case and generated case with the same condition (segmentation and/or ROI)

In [ ]:
import os
import numpy as np
import torch
import sys
import json
import nibabel as nib
from glob import glob
from tqdm import tqdm
sys.path.append(".")
from test_utils import absolute_mean_error
from test_utils import get_data_loader, get_BraTS_data_loader

### CT

In [ ]:
def get_CT_dataloader(clip_min, clip_max, fake_data_dir, pad_or_crop, load_only_fake):
    # Real data loader
    image_key = 'image'

    original_data_dir = "../../HnN_cancer_data/HnN_cancer_data_1_1_1_256_256_256"
    # Load Training and Test data split
    with open("../../HnN_cancer_data/HnN_cancer_data_1_1_1_256_256_256/data_split.json", 'r') as f:
        data_split = json.load(f)
        training = data_split['training']

    # Loading all real cases to memory
    real_image_files = []
    for file_name in training:
        real_image_files.append(file_name['image'])
    original_data_list = [{"image": os.path.join(original_data_dir, img)} for img in real_image_files]
    print(f"real original_data_list: {original_data_list[0]}")


    fake_image_files = []
    black_list = []
    for img in real_image_files:
        img = img.split('/')[-1]
        fake_name = img.replace('.nii.gz', '_CT_n0.nii.gz')
        fake_path = os.path.join(fake_data_dir, fake_name)
 
        if os.path.exists(fake_path):
            fake_image_files.append(fake_path)
        else:
            black_list.append(img)
    data_list = [{"image": img} for img in fake_image_files]
    print(f"fake data_list: {data_list[0]}")
    fake_data_loader = get_data_loader(image_key, clip_min, clip_max, data_list, pad_or_crop=pad_or_crop, two_loaders=False, cache_rate=[0])
    
    if not load_only_fake:        
        real_data_loader = get_data_loader(image_key, clip_min, clip_max, original_data_list, pad_or_crop=pad_or_crop, two_loaders=False, cache_rate=[0]) # change 0 to 1
        return real_data_loader, fake_data_loader, black_list
    else:
        return fake_data_loader, black_list
clip_min = -200
clip_max = 200
load_only_fake = False
pad_or_crop=False
fake_data_dir = "../results/Synthetic_Datasets/Whole_scans/Bone_segmentation/200/DPM++_2M"
real_data_loader, fake_data_loader, black_list = get_CT_dataloader(clip_min, clip_max, fake_data_dir, pad_or_crop=pad_or_crop, load_only_fake=load_only_fake)


In [ ]:
import warnings
import numpy as np
import nibabel as nib

real_iter = iter(real_data_loader)
pbar = tqdm(enumerate(fake_data_loader), total=len(fake_data_loader))
mae_results = {}
for fake_batch_idx, fake_batch in pbar: # iterate over fake cases 
    real_batch = next(real_iter)
    case_name = fake_batch['image_meta_dict']['filename_or_obj'][0].split('/')[-1].replace('_CT_n0.nii.gz', '.nii.gz')
    if case_name in black_list:
        continue
    else:
        y_fake = fake_batch['image']
        y_real = real_batch['image']
        if case_name != real_batch['image_meta_dict']['filename_or_obj'][0].split('/')[-1]:
            warnings.warn(f"The cases don't match. Double check: {case_name} \n {real_batch['image_meta_dict']['filename_or_obj'][0].split('/')[-1]}", SyntaxWarning)
        else:
            try:
                mae_here = absolute_mean_error(y_real[:,:,:-1,:-1,:-1], y_fake)
            except:
                # Some cases don't have the same orientation. Therefore, they need to be flipped
                y_fake = y_fake.permute(0, 1, 2, 4, 3)
                y_fake = torch.flip(y_fake, dims=[4])
                mae_here = absolute_mean_error(y_real[:,:,:-1,:-1,:-1], y_fake)
            pbar.set_postfix({"mae_here": float(mae_here)})
        
            mae_results[case_name.replace('.nii.gz', '')] = float(mae_here)
            
print(mae_results)
with open("./trash/mae.json", "w") as json_file:
    json.dump(mae_results, json_file, indent=4)

#### Mean

In [ ]:
import json
import os
import numpy as np

def compute_mean(data):
    all_mae_values = []
    for key in data:
        all_mae_values.append(data[key])
    all_mae_values = np.array(all_mae_values)
    # Compute mean and standard deviation
    mean = np.mean(all_mae_values)
    std_dev = np.std(all_mae_values)
    return mean, std_dev

results = []
root_path = "./metrics/MAE"
for modality in os.listdir(root_path):
    if modality=="MRI":
        exps_path = os.path.join(root_path, modality, 'Tumour_generation')
        for exp in os.listdir(exps_path):
            exp_path = os.path.join(exps_path, exp)       
            for sampler in os.listdir(exp_path):
                mae_json = os.path.join(exp_path, sampler, 'MAE.json')
                with open(mae_json, 'r') as file:
                    data = json.load(file)
                mean, std_dev = compute_mean(data)
                #print(f"{mae_json.split('MAE/')[-1]}: {mean}")
                #print(f"{mae_json.split('MAE/')[-1]}: {std_dev}")
                #print("#############")
                result = {
                            "Model": exp,
                            "Sampling Method": sampler,
                            "MAE ": f"{mean}±{std_dev}"
                        }
                #results.append(result)
    elif modality=="Whole_scans":
        tasks_path = os.path.join(root_path, modality)
        for task in os.listdir(tasks_path):
            sub_tasks_path = os.path.join(tasks_path, task)
            if task=="Tumour_inpaint":
                for noise_type in os.listdir(sub_tasks_path):
                    noise_type_path = os.path.join(sub_tasks_path, noise_type)
                    for hu_value in os.listdir(noise_type_path):
                        hu_value_path = os.path.join(noise_type_path, hu_value)
                        for sampler in os.listdir(hu_value_path):
                            mae_json = os.path.join(hu_value_path, sampler, 'MAE.json')
                            with open(mae_json, 'r') as file:
                                data = json.load(file)
                            mean, std_dev = compute_mean(data)
                            #print(f"{mae_json.split('MAE/')[-1]}: {mean}")
                            #print(f"{mae_json.split('MAE/')[-1]}: {std_dev}")
                            #print("#############")
                            result = {
                                "Model": hu_value,
                                "Sampling Method": sampler,
                                "MAE ": f"{mean}±{std_dev}"
                            }
                            results.append(result)

            else:
                for sub_task in os.listdir(sub_tasks_path):
                    sub_task_path = os.path.join(sub_tasks_path, sub_task)
                    for sampler in os.listdir(sub_task_path):
                        mae_json = os.path.join(sub_task_path, sampler, 'MAE.json')
                        with open(mae_json, 'r') as file:
                            data = json.load(file)
                        mean, std_dev = compute_mean(data)
                        print(f"{mae_json.split('MAE/')[-1]}: {mean}")
                        print(f"{mae_json.split('MAE/')[-1]}: {std_dev}")
                        print("#############")
                        result = {
                            "Model": sub_task,
                            "Sampling Method": sampler,
                            "MAE ": f"{mean}±{std_dev}"
                        }
                        #results.append(result)


        
print(results)
# Create a DataFrame from the results
df = pd.DataFrame(results)

# Save the DataFrame to an Excel file
df.to_excel('./metrics/MAE/resume_mae_200_CT.xlsx', index=False)

print("./metrics/MAE/resume_mae_200_CT.xlsx")


In [ ]:
mae_json = './metrics/MAE/Dataset982_HnN_GAN/MAE.json'
with open(mae_json, 'r') as file:
    data = json.load(file)
mean, std_dev = compute_mean(data)
print(f"{mae_json.split('MAE/')[-1]}: {mean}")
print(f"{mae_json.split('MAE/')[-1]}: {std_dev}")
print("##########")
mae_json = './metrics/MAE/Dataset996_BraTS_GAN/MAE.json'
with open(mae_json, 'r') as file:
    data = json.load(file)
mean, std_dev = compute_mean(data)
print(f"{mae_json.split('MAE/')[-1]}: {mean}")
print(f"{mae_json.split('MAE/')[-1]}: {std_dev}")

### BraTS

In [ ]:
def get_MRI_dataloader(fake_data_dir):
    # Real data loader
    original_data_dir = "../../../brats2023/ASNR-MICCAI-BraTS2023-GLI-Challenge-TrainingData"
    # Load Training and Test data split
    with open("../../../brats2023/BraTS2023_GLI_data_split.json", 'r') as f:
        data_split = json.load(f)
        training = data_split['training']

    # Loading all real cases to memory
    real_image_files = []
    for file_name in training:
        real_image_files.append(file_name['t1c'])
    original_data_list = [{"image": os.path.join(original_data_dir, img)} for img in real_image_files]
    print(f"real original_data_list: {original_data_list[0]}")

    fake_image_files = []
    black_list = []
    for img in real_image_files:
        img = img.split('/')[-1]
        fake_name = img.replace('.nii.gz', '_n0.nii.gz')
        fake_path = os.path.join(fake_data_dir, fake_name)
        if os.path.exists(fake_path):
            fake_image_files.append(fake_path)
        else:
            black_list.append(img)
    data_list = [{"image": img} for img in fake_image_files]
    print(f"fake data_list: {data_list[0]}")

    fake_data_loader = get_BraTS_data_loader(in_keys="image", data_list=data_list, cache_rate=0, apply_quantile=False)   
    real_data_loader = get_BraTS_data_loader(in_keys="image", data_list=original_data_list, cache_rate=0, apply_quantile=True)
    return real_data_loader, fake_data_loader, black_list



fake_data_dir = "../results/Synthetic_Datasets/MRI/Tumour_generation/concat_cond/DPM++_2M"
real_data_loader, fake_data_loader, black_list = get_MRI_dataloader(fake_data_dir=fake_data_dir)



In [ ]:
import warnings
import numpy as np
import nibabel as nib

real_iter = iter(real_data_loader)
pbar = tqdm(enumerate(fake_data_loader), total=len(fake_data_loader))
mae_results = {}
for fake_batch_idx, fake_batch in pbar: # iterate over fake cases 
    real_batch = next(real_iter)
    case_name = fake_batch['image_meta_dict']['filename_or_obj'][0].split('/')[-1].replace('_n0.nii.gz', '.nii.gz')
    if case_name in black_list:
        continue
    else:
        y_fake = fake_batch['image']
        y_real = real_batch['image']
        if case_name != real_batch['image_meta_dict']['filename_or_obj'][0].split('/')[-1]:
            warnings.warn(f"The cases don't match. Double check: {case_name} \n {real_batch['image_meta_dict']['filename_or_obj'][0].split('/')[-1]}", SyntaxWarning)
        else:
            mae_here = absolute_mean_error(y_real, y_fake)
            pbar.set_postfix({"mae_here": float(mae_here)})
            mae_results[case_name.replace('.nii.gz', '')] = float(mae_here)
            break
print(mae_results)
with open("./trash/mae.json", "w") as json_file:
    json.dump(mae_results, json_file, indent=4)